In [0]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor

gymlogger.set_level(40) #error only

import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay
from pyvirtualdisplay import Display

display = Display(visible=0, size=(1400, 900))
display.start()

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


In [0]:
ENV = wrap_env(gym.make('MountainCar-v0'))
ENV.reset()

array([-0.42595915,  0.        ])

In [0]:
BIN_SZIE = [10, 10]
SINGLE_BIN_VALUE = (ENV.observation_space.high - ENV.observation_space.low) / BIN_SZIE

LEARNING_RATE = 0.1
DISCOUNT = 0.95
EPISODE = 25000

EPSILON = 1
START_EPSILON_DECAY = 1
END_EPSILON_DECAY = EPISODE // 2
EPSILON_DECAY_VALUE = EPSILON / (END_EPSILON_DECAY - START_EPSILON_DECAY)

In [0]:
Q_TABLE = np.random.uniform(low = -2, high = 0, size = (BIN_SZIE + [ENV.action_space.n]))

In [0]:
def Get_Discrete_Value(STATE):
  DISCRETE_STATE = (STATE - ENV.observation_space.low) / SINGLE_BIN_VALUE

  return tuple(DISCRETE_STATE.astype(np.int))

In [0]:
DONE = False

for ESP in range(EPISODE):
  DISCRETE_STATE = Get_Discrete_Value(ENV.reset())
  DONE = False

  while not DONE:
    if np.random.random() > EPSILON:
      ACTION = np.argmax(Q_TABLE[DISCRETE_STATE])
    else:
      ACTION = np.random.randint(0, ENV.action_space.n)

    NEW_STATE, REWARD, DONE, _ = ENV.step(ACTION)

    NEW_DISCRETE_STATE = Get_Discrete_Value(NEW_STATE)

    if not DONE:
      MAX_FUTURE_Q = np.max(Q_TABLE[NEW_DISCRETE_STATE])
      CURRENT_Q = Q_TABLE[DISCRETE_STATE + (ACTION,)]
      NEW_Q = (1 - LEARNING_RATE) * CURRENT_Q + LEARNING_RATE * (REWARD + DISCOUNT * MAX_FUTURE_Q)
      Q_TABLE[DISCRETE_STATE + (ACTION,)] = NEW_Q

    elif NEW_STATE[0] >= ENV.goal_position:
      Q_TABLE[DISCRETE_STATE + (ACTION,)] = 0

    DISCRETE_STATE = NEW_DISCRETE_STATE

  if END_EPSILON_DECAY >= ESP >= START_EPSILON_DECAY:
    EPSILON -= EPSILON_DECAY_VALUE

ENV.close()
show_video()
  